In [9]:
import cv2
import argparse
import numpy as np
import utils

In [10]:
%%time
# read input image
list_images = utils.get_files_by_ext("object-detection-opencv/", "jpg")
classes_file = "object-detection-opencv/yolov3.txt"
weights_file = "object-detection-opencv/yolov3.weights"
conf_file = "object-detection-opencv/yolov3.cfg"

conf_threshold = 0.5
nms_threshold = 0.4

Width = list()
Height = list()
for image_path in list_images:
    image = cv2.imread(image_path)

    Width.append(image.shape[1])
    Height.append(image.shape[0])

scale = 0.00392

# read class names from text file
classes = None
with open(classes_file, 'r') as f:
    classes = [line.strip() for line in f.readlines()]

# generate different colors for different classes 
COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

# create input blob 
# prepares the input image to run through the deep neural network
blob = utils.image_to_matrix(list_images)

# read pre-trained model and config file
# reads the weights and config file and creates the network.
net = cv2.dnn.readNet(weights_file, conf_file)

# set input blob for the network
net.setInput(blob)

# run inference through the network
# and gather predictions from output layers
outs = net.forward(utils.get_output_layers(net))


CPU times: user 11.4 s, sys: 190 ms, total: 11.6 s
Wall time: 7.1 s


In [11]:
%%time
# identify objects into the list of images analyzed by the network
dict_obj_detected = {}
dict_obj_detected = utils.detect_object(outs, list_images, Width, Height)

# identify pictures with at least one perso
dict_danger = utils.detect_danger(dict_obj_detected)
for image_name, is_dangerous in dict_danger.items():
    print("Is", image_name, "dangerous ? \t\t\t" , is_dangerous)

Is object-detection-opencv/344.jpg dangerous ? 			 True
Is object-detection-opencv/dog.jpg dangerous ? 			 False
Is object-detection-opencv/object-detection.jpg dangerous ? 			 False
Is object-detection-opencv/test.jpg dangerous ? 			 True
Is object-detection-opencv/test_no_person.jpg dangerous ? 			 False
CPU times: user 477 ms, sys: 4.04 ms, total: 481 ms
Wall time: 478 ms


In [12]:
# Draw bounding boxes and save images
for image_path, image_items in dict_obj_detected.items():
    utils.get_bounding_box(image_path, image_items, classes, COLORS, conf_threshold, nms_threshold)